In [1]:
import pickle

In [2]:
import os
from tqdm import tqdm
from typing import Dict
from collections import defaultdict
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def numLines(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1
def loadData(filename, max_points):
    file_len = numLines(filename)
    f = open(filename, 'r')
    inputs = []
    outputs = []
    for i in tqdm(range(file_len)):
        if i == max_points:
            break
        line = f.readline()
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        inputs.append(line[0])
        outputs.append(line[1])
    data = {'inputs': inputs, 'outputs': outputs}
    return data
        
def load_entity_strings(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

def get_entity_wd_id_dict(filename):
    out = {}
    f = open(filename, 'r')
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        out[line[1]] = line[0]
    return out
    

def create_filter_dict(data) -> Dict[str, int]:
    filter_dict = defaultdict(list)
    for input, output in zip(data["inputs"], data["outputs"]):
        filter_dict[input].append(output)
    return filter_dict

def getAllFilteringEntities(input, filter_dicts):
    entities = []
    splits = ['train', 'test', 'valid']
    for s in splits:
        entities.extend(filter_dicts[s][input])
    return list(set(entities))

def wikidata_link_from_id(id):
    uri = 'https://www.wikidata.org/wiki/' + id
    return uri

In [3]:
dataset_name = 'MetaQA_half'
entity_strings = load_entity_strings(os.path.join("data", dataset_name, "entity_strings.txt"))

In [4]:
entity_strings_set = set(entity_strings)

In [5]:
data = {}
splits = ['train', 'valid', 'test']
dataset_name = 'MetaQA_half'
for split in splits:
    data[split] = loadData(os.path.join('data', dataset_name, split + '.txt'), -1)

100%|██████████| 8000/8000 [00:00<00:00, 679225.76it/s]


In [7]:
filter_dicts = {}
splits = ['train', 'valid', 'test']
for split in splits:
    filter_dicts[split] = create_filter_dict(data[split])

In [175]:
# fname = 'scores.pickle'
fname = 'scores/scores_temp.pickle'
# fname = 'scores_500_base_trie.pickle'
scores_data = pickle.load(open(fname, 'rb'))

In [176]:
predictions_scores_dicts = []
for pred, score in tqdm(zip(scores_data['prediction_strings'], scores_data['scores'])):
    ps_pair = (pred, score)
    # remove predictions that are not entities
    ps_pairs = [ps_pair]
    ps_dict_only_entities = defaultdict(list)
    for ps in ps_pairs:
        if True:
#         if ps[0] in entity_strings_set:
            ps_dict_only_entities[ps[0]] = ps[1]
    predictions_scores_dicts.append(ps_dict_only_entities)

14274it [00:00, 103349.96it/s]


In [177]:
scores_data['target_strings'][0]

['Drama', 'Music']

In [178]:
predictions_scores_dicts[0]

defaultdict(list, {'Drama': -0.5109100341796875})

In [179]:
import numpy as np
# no filtering for QA
predictions_filtered = predictions_scores_dicts


In [180]:
count = {}
reciprocal_ranks = 0.0
k_list = [1,3,10]
for k in k_list:
    count[k] = 0
num_small_arrs = 0
for i in range(len(predictions_filtered)):
    targets = scores_data['target_strings'][i]
    ps_dict = predictions_filtered[i]
    ps_sorted = sorted(ps_dict.items(), key=lambda item: -item[1])
#     print(ps_dict)
    if len(ps_dict) == 0:
        preds = []
    else:
        preds = [x[0] for x in ps_sorted]
    pred = preds[0]
    
    if pred in targets:
        rank = preds.index(pred) + 1
        reciprocal_ranks += 1./rank
for k in k_list:
    hits_at_k = count[k]/len(predictions_filtered)
    print('hits@{}'.format(k), hits_at_k)
print('mrr', reciprocal_ranks/len(predictions_filtered))
print(num_small_arrs/len(predictions_filtered), 'were <10 length preds array without answer')

hits@1 0.0
hits@3 0.0
hits@10 0.0
mrr 0.4658119658119658
0.0 were <10 length preds array without answer


In [172]:
id = 13
inputs = scores_data['input_strings'][id]
preds = predictions_filtered[id]
preds = sorted(preds.items(), key=lambda item: -item[1])
target = scores_data['target_strings'][id]
print(inputs, 'Target:', target)
preds[0], target

predict answer: Rafael Yglesias | who starred in the movies written by NE | Target: ['Isabella Rossellini', 'Ben Kingsley', 'Jet Li', 'Jeff Bridges', 'Rosie Perez', 'Stuart Wilson', 'Sigourney Weaver']


(('Christopher Walken', -2.131134033203125),
 ['Isabella Rossellini',
  'Ben Kingsley',
  'Jet Li',
  'Jeff Bridges',
  'Rosie Perez',
  'Stuart Wilson',
  'Sigourney Weaver'])

In [181]:
def getHeadEntity(input):
    x = input.split(':')[1][1:]
    ent = x.split('|')[0][:-1]
    return ent

In [183]:
# see if answer is head entity
count = 0
for id in range(len(predictions_filtered)):
    input = scores_data['input_strings'][id]
    head = getHeadEntity(input)
    pred = scores_data['prediction_strings'][id]
    if head == pred:
        count += 1
        print(input)
count/len(predictions_filtered)

0.0

In [113]:
# only head/tails
count = 0
for id in range(60,120, 2):
    inputs = scores_data['input_strings'][id]
    preds = predictions_filtered[id]
    preds = sorted(preds.items(), key=lambda item: -item[1])
    target = scores_data['target_strings'][id]
    pred1 = preds[0][0]
    if pred1 == target:
        print(int(id/2), inputs, pred1)
        count += 1
'count', count

('count', 0)

In [419]:
%%html
print("<a href='your_url_here'>Showing Text</a>")


In [364]:
e2wdid['pakistan']

'Q4121082'

In [174]:
sequences = ['english', 'english language', 'french']
t = Trie(sequences)

In [178]:
t.get('x')

[]